# **Image-to-Image Translation with cGAN**

In [ ]:
# Install necessary libraries
!pip install tensorflow tensorflow-datasets matplotlib

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# Load the CIFAR-10 dataset
def load_cifar10_dataset():
    (train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()
    return train_images

# Preprocess the images
def preprocess_image(image):
    image = tf.image.resize(image, [64, 64])  # Resize to 64x64 for faster processing
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

# Load and preprocess dataset
def load_dataset(images):
    images = np.array(images)
    images = np.array([preprocess_image(img) for img in images])
    return images

# Create the generator model
def build_generator():
    inputs = layers.Input(shape=[64, 64, 3])

    # Downsampling
    down_stack = [
        layers.Conv2D(64, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2D(128, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2D(256, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, (4, 4), strides=2, padding='same', activation='relu'),
    ]

    # Upsampling
    up_stack = [
        layers.Conv2DTranspose(256, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(128, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(64, (4, 4), strides=2, padding='same', activation='relu'),
    ]

    last = layers.Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh')

    x = inputs
    for down in down_stack:
        x = down(x)

    for up in up_stack:
        x = up(x)

    return tf.keras.Model(inputs=inputs, outputs=last(x))

# Create the discriminator model
def build_discriminator():
    inputs = layers.Input(shape=[64, 64, 3])

    x = layers.Conv2D(64, (4, 4), strides=2, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(128, (4, 4), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (4, 4), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, (4, 4), strides=2, padding='same', activation='relu')(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

# Define the training loop
@tf.function
def train_step(real_images, generator, discriminator, generator_optimizer, discriminator_optimizer):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_images = generator(real_images, training=True)

        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)

        gen_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)
        disc_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output) + \
                    tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Define the training loop
def train(dataset, epochs=3):
    generator = build_generator()
    discriminator = build_discriminator()

    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    for epoch in range(epochs):
        for real_images in dataset:
            gen_loss, disc_loss = train_step(real_images, generator, discriminator, generator_optimizer, discriminator_optimizer)

            print(f'Epoch: {epoch + 1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')

# Load dataset and train
cifar10_images = load_cifar10_dataset()
dataset = load_dataset(cifar10_images[:2000])  # Use only 2000 images for faster processing

# Create a TensorFlow dataset from images
tf_dataset = tf.data.Dataset.from_tensor_slices(dataset).batch(32)

# Start training (change epochs as needed)
train(tf_dataset, epochs=3)  # Change epochs as needed


Epoch: 1, Gen Loss: 0.6931400299072266, Disc Loss: 1.3854243755340576
Epoch: 1, Gen Loss: 0.6925402879714966, Disc Loss: 1.2671738862991333
Epoch: 1, Gen Loss: 0.6923026442527771, Disc Loss: 1.1250510215759277
Epoch: 1, Gen Loss: 0.692035436630249, Disc Loss: 0.9033427238464355
Epoch: 1, Gen Loss: 0.688727855682373, Disc Loss: 0.7822825908660889
Epoch: 1, Gen Loss: 0.6208856701850891, Disc Loss: 0.780074954032898
Epoch: 1, Gen Loss: 0.3835853934288025, Disc Loss: 1.1713273525238037
Epoch: 1, Gen Loss: 0.17311757802963257, Disc Loss: 1.95594322681427
Epoch: 1, Gen Loss: 0.15428584814071655, Disc Loss: 2.068049907684326
Epoch: 1, Gen Loss: 0.4521525800228119, Disc Loss: 1.2824029922485352
Epoch: 1, Gen Loss: 0.7388051152229309, Disc Loss: 1.0359723567962646
Epoch: 1, Gen Loss: 0.9081608057022095, Disc Loss: 0.7796381115913391
Epoch: 1, Gen Loss: 0.9940921068191528, Disc Loss: 0.5932983160018921
Epoch: 1, Gen Loss: 1.4126330614089966, Disc Loss: 0.3583661913871765
Epoch: 1, Gen Loss: 2.14

In [ ]:
# Install necessary libraries
!pip install tensorflow tensorflow-datasets matplotlib

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# Load the maps dataset (or any paired dataset)
def load_maps_dataset():
    dataset, _ = tfds.load('oxford_iiit_pet:3.*.*', with_info=True, as_supervised=True)
    return dataset['train'], dataset['test']

# Preprocess the images
def preprocess_image(image, label):
    image = tf.image.resize(image, [64, 64])  # Resize to 64x64 for faster processing
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    label = tf.image.resize(label, [64, 64])
    label = (label - 127.5) / 127.5  # Normalize label to [-1, 1]
    return image, label

# Load and preprocess dataset
def load_dataset(dataset):
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    return dataset

# Build the generator model (consider adding skip connections)
def build_generator():
    inputs = layers.Input(shape=[64, 64, 3])

    # Downsampling
    down_stack = [
        layers.Conv2D(64, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2D(128, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2D(256, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, (4, 4), strides=2, padding='same', activation='relu'),
    ]

    # Upsampling
    up_stack = [
        layers.Conv2DTranspose(256, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(128, (4, 4), strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(64, (4, 4), strides=2, padding='same', activation='relu'),
    ]

    last = layers.Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh')

    x = inputs
    for down in down_stack:
        x = down(x)

    for up in up_stack:
        x = up(x)

    return tf.keras.Model(inputs=inputs, outputs=last(x))

# Build the discriminator model
def build_discriminator():
    inputs = layers.Input(shape=[64, 64, 3])

    x = layers.Conv2D(64, (4, 4), strides=2, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(128, (4, 4), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (4, 4), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, (4, 4), strides=2, padding='same', activation='relu')(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

# Define the L1 loss
def compute_l1_loss(real_labels, fake_images):
    return tf.reduce_mean(tf.abs(real_labels - fake_images))

# Define the training loop
@tf.function
def train_step(real_images, real_labels, generator, discriminator, generator_optimizer, discriminator_optimizer):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_images = generator(real_images, training=True)

        real_output = discriminator(real_labels, training=True)
        fake_output = discriminator(fake_images, training=True)

        # Define generator and discriminator losses
        gen_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output) + compute_l1_loss(real_labels, fake_images)
        disc_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output) + \
                    tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Define the training loop
def train(dataset, epochs=3):  # Increase epochs as needed
    generator = build_generator()
    discriminator = build_discriminator()

    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    for epoch in range(epochs):
        for real_images, real_labels in dataset:
            gen_loss, disc_loss = train_step(real_images, real_labels, generator, discriminator, generator_optimizer, discriminator_optimizer)

            print(f'Epoch: {epoch + 1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')

# Load dataset and train
train_data, test_data = load_maps_dataset()
tf_dataset = load_dataset(train_data)

# Start training
train(tf_dataset, epochs=3)  # Adjust epochs as needed


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]